<a href="https://colab.research.google.com/github/CanKeles5/ObjectDetection/blob/main/Detectron2_Faster_R_CNN_VisDrone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install dependencies: 
!pip install pyyaml==5.1
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
# opencv is pre-installed on colab

     |████████████████████████████████| 276kB 18.0MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.1-cp37-cp37m-linux_x86_64.whl size=44074 sha256=ebb18a13de6399121736ae2a5cc6f9b299e41ee232f6cca71580d2438ce811a3
  Stored in directory: /root/.cache/pip/wheels/ad/56/bc/1522f864feb2a358ea6f1a92b4798d69ac783a28e80567a18b
Successfully built pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
1.7.1+cu101 True
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [2]:
# install detectron2: (Colab has CUDA 10.1 + torch 1.7)
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
import torch
assert torch.__version__.startswith("1.7")
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.7/index.html
exit(0)  # After installation, you need to "restart runtime" in Colab. This line can also restart runtime

Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.7/index.html
     |████████████████████████████████| 6.8MB 689kB/s 
     |████████████████████████████████| 51kB 7.4MB/s 
     |████████████████████████████████| 2.2MB 24.2MB/s 
  Created wheel for fvcore: filename=fvcore-0.1.3.post20210227-cp37-none-any.whl size=59001 sha256=3cb85a0d00ccdc97946d929267b92021409b63a8deb1b60360dab274a6a99118
  Stored in directory: /root/.cache/pip/wheels/89/4b/64/716bf9925b2bb322cf85281e7d34d36542847c544a85182d05
  Created wheel for iopath: filename=iopath-0.1.4-cp37-none-any.whl size=15571 sha256=2e99ae46a16e1efaae578efa70268337993b7c8d85ab350a17943a6fe0227251
  Stored in directory: /root/.cache/pip/wheels/54/3b/fb/25b60ef5989e3f5fd76fdc575c922e6e59066ddbd1b44756a9
Successfully built fvcore iopath
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: Pillow 7.0.0
    Uninstalling 

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import random
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.structures import BoxMode
from google.colab.patches import cv2_imshow
from detectron2.utils.visualizer import ColorMode

** fvcore version of PathManager will be deprecated soon. **
** Please migrate to the version in iopath repo. **
https://github.com/facebookresearch/iopath 

** fvcore version of PathManager will be deprecated soon. **
** Please migrate to the version in iopath repo. **
https://github.com/facebookresearch/iopath 



In [3]:
!unzip /content/drive/MyDrive/VisDrone2019-DET-train.zip -d /content/

Görüntülenen çıkış son 5000 satıra kısaltıldı.
 extracting: /content/VisDrone2019-DET-train/images/9999937_00000_d_0000140.jpg  
 extracting: /content/VisDrone2019-DET-train/images/9999937_00000_d_0000141.jpg  
 extracting: /content/VisDrone2019-DET-train/images/9999937_00000_d_0000142.jpg  
 extracting: /content/VisDrone2019-DET-train/images/9999937_00000_d_0000143.jpg  
 extracting: /content/VisDrone2019-DET-train/images/9999937_00000_d_0000144.jpg  
  inflating: /content/VisDrone2019-DET-train/images/9999937_00000_d_0000145.jpg  
 extracting: /content/VisDrone2019-DET-train/images/9999937_00000_d_0000146.jpg  
 extracting: /content/VisDrone2019-DET-train/images/9999937_00000_d_0000147.jpg  
 extracting: /content/VisDrone2019-DET-train/images/9999937_00000_d_0000148.jpg  
 extracting: /content/VisDrone2019-DET-train/images/9999937_00000_d_0000149.jpg  
  inflating: /content/VisDrone2019-DET-train/images/9999937_00000_d_0000150.jpg  
  inflating: /content/VisDrone2019-DET-train/images

In [4]:
!unzip /content/drive/MyDrive/VisDrone2019-DET-val.zip -d /content/

Archive:  /content/drive/MyDrive/VisDrone2019-DET-val.zip
   creating: /content/VisDrone2019-DET-val/
  inflating: /content/VisDrone2019-DET-val/.DS_Store  
   creating: /content/VisDrone2019-DET-val/annotations/
  inflating: /content/VisDrone2019-DET-val/annotations/0000001_02999_d_0000005.txt  
  inflating: /content/VisDrone2019-DET-val/annotations/0000001_03499_d_0000006.txt  
  inflating: /content/VisDrone2019-DET-val/annotations/0000001_03999_d_0000007.txt  
  inflating: /content/VisDrone2019-DET-val/annotations/0000001_04527_d_0000008.txt  
  inflating: /content/VisDrone2019-DET-val/annotations/0000001_05249_d_0000009.txt  
  inflating: /content/VisDrone2019-DET-val/annotations/0000001_05499_d_0000010.txt  
  inflating: /content/VisDrone2019-DET-val/annotations/0000001_05999_d_0000011.txt  
  inflating: /content/VisDrone2019-DET-val/annotations/0000001_07999_d_0000012.txt  
  inflating: /content/VisDrone2019-DET-val/annotations/0000001_08414_d_0000013.txt  
  inflating: /content/

In [5]:
#/content/VisDrone2019-DET-train/annotations
def get_visdrone_dicts(img_path = "/content/VisDrone2019-DET-train/images", annot_path = "/content/VisDrone2019-DET-train/annotations" ):
  dataset_dicts = []
  for path, subdirs, files in os.walk(img_path):
   for filename in files:
     record = {}
     
     
     img_p = os.path.join(path, filename)
     anot_p = os.path.join(annot_path, filename[:-4] + '.txt')

     h, w = cv2.imread(img_p).shape[:2]

     record["file_name"] = img_p
     record["image_id"] = filename
     record["height"] = h
     record["width"] = w

     objs = []

     with open(anot_p) as fp:
       line = fp.readline()
       while line:
         line = line.replace("\n","") 
         vals = line.split (",")
         id = int(vals[5])
         
         if id==0 or id == 11:
           id=0
         elif id==1 or id==2:
           id=1
         else:
           id=2    
        
         b_left, b_top, b_width, b_height = list(map(float,vals[:4]))
         b_right, b_bottom = b_left + b_width, b_top - b_height
         
         obj = {
            "bbox": [b_left, b_top, b_width, b_height],
            "bbox_mode": BoxMode.XYWH_ABS,
            "segmentation": [],
            "category_id": id,
         }
         objs.append(obj)
         line = fp.readline()
     record["annotations"] = objs
     dataset_dicts.append(record)
  
  return dataset_dicts

In [6]:
DatasetCatalog.register("train_set",  lambda d=_: get_visdrone_dicts("/content/VisDrone2019-DET-train/images", "/content/VisDrone2019-DET-train/annotations"))
MetadataCatalog.get("train_set").set(thing_classes =["Others", "Person", "Vehicle"])
MetadataCatalog.get("train_set").thing_colors = [(0,255,0), (255,0,0), (0,0,255)]

DatasetCatalog.register("val_set",  lambda d=_: get_visdrone_dicts("/content/VisDrone2019-DET-val/images", "/content/VisDrone2019-DET-val/annotations"))
MetadataCatalog.get("val_set").set(thing_classes =["Others", "Person", "Vehicle"])
MetadataCatalog.get("val_set").thing_colors = [(0,255,0), (255,0,0), (0,0,255)]

In [7]:
train_metadata = MetadataCatalog.get("train_set")
val_metadata = MetadataCatalog.get("val_set")

In [8]:
val_metadata

Metadata(name='val_set', thing_classes=['Others', 'Person', 'Vehicle'], thing_colors=[(0, 255, 0), (255, 0, 0), (0, 0, 255)])

In [9]:
train_set_dicts = get_visdrone_dicts("/content/VisDrone2019-DET-train/images", "/content/VisDrone2019-DET-train/annotations")
val_set_dicts = get_visdrone_dicts("/content/VisDrone2019-DET-val/images", "/content/VisDrone2019-DET-val/annotations")

In [10]:
val_set_dicts[0]

{'annotations': [{'bbox': [1258.0, 574.0, 102.0, 83.0],
   'bbox_mode': <BoxMode.XYWH_ABS: 1>,
   'category_id': 2,
   'segmentation': []},
  {'bbox': [1337.0, 603.0, 18.0, 36.0],
   'bbox_mode': <BoxMode.XYWH_ABS: 1>,
   'category_id': 1,
   'segmentation': []},
  {'bbox': [1015.0, 727.0, 16.0, 36.0],
   'bbox_mode': <BoxMode.XYWH_ABS: 1>,
   'category_id': 1,
   'segmentation': []},
  {'bbox': [991.0, 707.0, 20.0, 53.0],
   'bbox_mode': <BoxMode.XYWH_ABS: 1>,
   'category_id': 1,
   'segmentation': []},
  {'bbox': [962.0, 729.0, 17.0, 33.0],
   'bbox_mode': <BoxMode.XYWH_ABS: 1>,
   'category_id': 1,
   'segmentation': []},
  {'bbox': [981.0, 748.0, 13.0, 17.0],
   'bbox_mode': <BoxMode.XYWH_ABS: 1>,
   'category_id': 1,
   'segmentation': []},
  {'bbox': [968.0, 700.0, 20.0, 43.0],
   'bbox_mode': <BoxMode.XYWH_ABS: 1>,
   'category_id': 1,
   'segmentation': []},
  {'bbox': [880.0, 677.0, 86.0, 88.0],
   'bbox_mode': <BoxMode.XYWH_ABS: 1>,
   'category_id': 2,
   'segmentation': []

In [11]:
import json
from pathlib import Path
from PIL import Image as PILImage
import IPython
import numpy as np
from math import trunc
import base64
from io import BytesIO

In [11]:
#annot_path = "/content/drive/MyDrive/visdronecoco.json"
#img_path = "/content/VisDrone2019-DET-train/images"

In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
import os

cfg = get_cfg()
cfg.merge_from_file(
    model_zoo.get_config_file("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml")
)
cfg.DATASETS.TRAIN = ("train_set",)
cfg.DATASETS.TEST = ("val_set",)  # no metrics implemented for this dataset
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.BASE_LR = 0.0003
cfg.SOLVER.MAX_ITER = (2000)
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = (512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

[03/01 17:23:52 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

model_final_68b088.pkl: 421MB [00:16, 26.2MB/s]                           
Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (4, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (4,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (12, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (12,) in the model! You might want to double check if this is expected.


[03/01 17:26:14 d2.engine.train_loop]: Starting training from iteration 0


/usr/local/lib/python3.7/dist-packages/detectron2/modeling/roi_heads/fast_rcnn.py:217: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:882.)
  num_fg = fg_inds.nonzero().numel()


[03/01 17:27:17 d2.utils.events]:  eta: 1:47:48  iter: 19  total_loss: 3.016  loss_cls: 1.461  loss_box_reg: 0.595  loss_rpn_cls: 0.6116  loss_rpn_loc: 0.3628  time: 3.2090  data_time: 0.0375  lr: 5.9943e-06  max_mem: 11202M
[03/01 17:28:24 d2.utils.events]:  eta: 1:47:43  iter: 39  total_loss: 2.703  loss_cls: 1.395  loss_box_reg: 0.6678  loss_rpn_cls: 0.3462  loss_rpn_loc: 0.3282  time: 3.2743  data_time: 0.0191  lr: 1.1988e-05  max_mem: 11268M
[03/01 17:29:36 d2.utils.events]:  eta: 1:50:22  iter: 59  total_loss: 2.611  loss_cls: 1.254  loss_box_reg: 0.6301  loss_rpn_cls: 0.3368  loss_rpn_loc: 0.3885  time: 3.3797  data_time: 0.0175  lr: 1.7982e-05  max_mem: 11538M
[03/01 17:30:42 d2.utils.events]:  eta: 1:48:03  iter: 79  total_loss: 2.261  loss_cls: 1.079  loss_box_reg: 0.6024  loss_rpn_cls: 0.1946  loss_rpn_loc: 0.3481  time: 3.3629  data_time: 0.0198  lr: 2.3976e-05  max_mem: 11538M
[03/01 17:31:48 d2.utils.events]:  eta: 1:46:45  iter: 99  total_loss: 2.063  loss_cls: 0.8783  l

In [ ]:
# Look at training curves in tensorboard:
%load_ext tensorboard
%tensorboard --logdir output

In [ ]:
!unzip /content/drive/MyDrive/VisDrone2019-DET-test-dev.zip -d /content/

In [ ]:
DatasetCatalog.register("datset10",  lambda d= _: get_visdrone_dicts("/content/images", "/content/annotations"))
MetadataCatalog.get("datset10").set(thing_classes =["Ignored", "Pedestrian", "People", "Bicycle", "Car", "Van", "Truck", "Tricycle", "awning-tricycle", "Bus", "Motor", "Others"])
MetadataCatalog.get("datset10").thing_colors = [(0,255,0), (255,0,0), (255,0,0), (0,0,255), (0,0,255), (0,0,255),(0,0,255),(0,0,255),(0,0,255),(0,0,255),(0,0,255),(0,255,0)]

In [ ]:
balloon_metadata = MetadataCatalog.get("datset10")

In [ ]:
dataset_dicts = get_visdrone_dicts("/content/images", "/content/annotations")

In [ ]:
print(cfg.OUTPUT_DIR)

In [ ]:
# Inference should use the config with parameters that are used in training
# cfg now already contains everything we've set previously. We changed it a little bit for inference:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "/content/output/model_final.pth")  # path to the model we just trained
#cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.4   # set a custom testing threshold
cfg.MODEL.RETINANET.SCORE_THRESH_TEST = 0.45
cfg.DATASETS.TEST = ( )
predictor = DefaultPredictor(cfg)

In [ ]:
for d in random.sample(dataset_dicts, 10):    
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1],
                   metadata=balloon_metadata, 
                   scale=1.5, 
                   instance_mode=ColorMode.SEGMENTATION   # remove the colors of unsegmented pixels
    )
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(v.get_image()[:, :, ::-1])


In [ ]:
#import the COCO Evaluator to use the COCO Metrics
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

#Call the COCO Evaluator function and pass the Validation Dataset
evaluator = COCOEvaluator("datset9", cfg, False, output_dir="/output2/")
val_loader = build_detection_test_loader(cfg, "datset9")

#Use the created predicted model in the previous step
inference_on_dataset(predictor.model, val_loader, evaluator)